In [1]:
from datasets import load_dataset

data = load_dataset("ashraq/financial-news-articles",split='train')

/home/apprenant/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/apprenant/.cache/huggingface/datasets/ashraq___parquet/ashraq--financial-news-articles-2c6ba3fd2690414f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [2]:
data

Dataset({
    features: ['title', 'text', 'url'],
    num_rows: 306242
})

In [3]:
import tensorflow as tf
from transformers import pipeline
from transformers import TFAutoModel
from transformers import AutoTokenizer


# model_checkpoint = "human-centered-summarization/financial-summarization-pegasus"

model_checkpoint = "google/mt5-small"
model = TFAutoModel.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


2023-03-27 08:38:35.148115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 08:38:35.542839: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 08:38:35.542854: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-27 08:38:36.782697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
model.summary()

Model: "tfmt5_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
Total params: 172,119,424
Trainable params: 172,119,424
Non-trainable params: 0
_________________________________________________________________


In [5]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [336, 259, 28387, 11807, 287, 62893, 295, 12507, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁', 'loved', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '!', '</s>']

In [7]:
max_input_length = 512
max_target_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_datasets = data.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/apprenant/.cache/huggingface/datasets/ashraq___parquet/ashraq--financial-news-articles-2c6ba3fd2690414f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-21133bb21f9d0737.arrow


In [9]:
import evaluate

rouge_score = evaluate.load("rouge")

In [10]:
generated_summary = "I absolutely loved reading the Hunger Games"
# "J'ai absolument adoré lire les Hunger Games"
reference_summary = "I loved reading the Hunger Games"
# "J'ai adoré lire les Hunger Games"

In [11]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [12]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/apprenant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(data[1]["text"]))

NEW YORK (Reuters) - European stocks closed lower on Tuesday, the first trading day of 2018, while Wall Street advanced and the U.S. dollar fell to its weakest in over three months against key currencies.
A trader works on the floor of the New York Stock Exchange shortly after the opening bell in New York, U.S., January 2, 2018.
REUTERS/Lucas Jackson MSCI’s gauge of stocks across the globe .MIWD PUS gained 0.61 percent.


In [14]:


def evaluate_baseline(dataset, metric):
    
    summaries = [three_sentence_summary(text) for text in dataset["text"]]
    return metric.compute(predictions=summaries, references=dataset["title"])


In [15]:
# import pandas as pd


# score = evaluate_baseline(data[0:4000], rouge_score)
# rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
# rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
# rouge_dict

In [16]:
data

Dataset({
    features: ['title', 'text', 'url'],
    num_rows: 306242
})

In [17]:
from datasets import Dataset
tokenized_datasets = data.map(preprocess_function, batched=True)
tokenized_data_train = tokenized_datasets[0:20000]
tokenized_data_validation = tokenized_datasets[20000:25000]

tokenized_data_test = tokenized_datasets[25000:35000]

tokenized_data_train=Dataset.from_dict(tokenized_data_train)
tokenized_data_validation=Dataset.from_dict(tokenized_data_validation)
tokenized_data_test=Dataset.from_dict(tokenized_data_test)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM,AutoConfig


config=AutoConfig.from_pretrained(model_checkpoint, output_attentions=True,output_hidden_states=True)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,config=config)

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [ ]:
model.summary()

Model: "tfmt5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
 lm_head (Dense)             multiple                  128057344 
                                                                 
Total params: 300,176,768
Trainable params: 300,176,768
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
dense_layer = Dense(units=512, activation='relu')



# Freeze the weights of the original model
for layer in model.layers[:-1]:
    layer.trainable = False





In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
features = [tokenized_datasets[i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'title': <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'BRIEF-Bigger Capital Fund Reports An 8 Pct Passive Stake In Akers Biosciences',
       b'Global Markets: Asia shares reach decade top on China data, dollar in doldrums'],
      dtype=object)>, 'text': <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'January 2, 2018 / 9:31 PM / Updated 8 minutes ago BRIEF-Bigger Capital Fund Reports An 8 Pct Passive Stake In Akers Biosciences Reuters Staff 1 Min Read \nJan 2 (Reuters) - Biosciences Inc: \n* BIGGER CAPITAL FUND LP REPORTS AN 8 PCT PASSIVE STAKE IN AKERS AS OF DEC 21, 2017 - SEC FILING Source text : ( bit.ly/2lHEjFS ) Further company coverage:',
       b"NEW YORK (Reuters) - European stocks closed lower on Tuesday, the first trading day of 2018, while Wall Street advanced and the U.S. dollar fell to its weakest in over three months against key currencies.\nA trader works on the floor of the New York Stock Exchange shortly after the opening bell in New York, U.S., January

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_data_train,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_data_validation,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

# Le nombre d'étapes d'entraînement est le nombre d'échantillons dans le jeu de données, divisé par la taille du batch,
# puis multiplié par le nombre total d'époques. Notez que le jeu de données tf_train_dataset est ici un tf.data.Dataset,
# et non le jeu de données original donc son len() est déjà num_samples // batch_size.
num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=1e-2,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Entraîner en mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:

callback =tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=8
)

Epoch 1/8
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


 882/1250 [====================>.........] - ETA: 46:07 - loss: 7.0521

KeyboardInterrupt: 